# 0. 이번 시간에는 'Perfect Idempotency(완벽한 멱등성)'에 대해서 실습을 진행해보려고 한다.

※Idempotency(멱등성)이란?

->연산을 여러 번 적용하더라도 결과가 달라지지 않는 성질(처음 결과와 동일하게 유지하는 성질)

->여러번 '실행'해도 기존 최신 '상태'가 변하지 않는 것.

->그래서 실무에서는 파이프라인이 100번 실패해서 100번 재실행하더라도, 최종 결과 테이블에는 단 한 건의 중복도, 단 한 건의 누락도 없어야 한다.



---



## 실습에 들어가기에 앞서... QnA

->※다소 긴 내용이지만, 그래도 순차적으로 흐름에 맡겨 한 번 읽어보면 나름 재밌다고 생각합니다. 😊👍



```
Q-1.🧐 소크라테스식 질문
1.Partial Failure (부분 실패): Spark로 S3에 데이터를 쓰고 있는데, 80% 정도 써진 상태에서 네트워크 장애로 작업이 죽었어. S3에는 이미 데이터 일부가 써졌겠지? 이때 네가 작업을 단순히 '다시 실행(Retry)'만 한다면, 최종 데이터는 어떻게 될까? 이걸 깨끗하게 해결할 수 있는 'Atomic Write(원자적 쓰기)' 전략은 무엇일까?

2.Backfill 시나리오: 비즈니스 로직이 바뀌어서 지난 1주일치 데이터를 다시 뽑아야 해. 이미 운영 DB에는 해당 날짜의 데이터가 들어있지. 기존 데이터를 수동으로 지우지 않고도, **파이프라인 실행만으로 기존 데이터를 안전하게 교체(Overwrite)**하려면 어떤 파티션 설계가 필요할까?

3.Unique Key의 부재: 로그 데이터(JSON)에는 보통 DB처럼 Primary Key가 명확하지 않아. 똑같은 로그가 두 번 들어왔을 때, 이게 '진짜 두 번 일어난 일'인지 '시스템 오류로 중복 수집된 것'인지 어떻게 판별하고 처리할래?
```

A-1(My Answer).


```
1. Partial Failure(부분적 실패)에서는 일단 일부 데이터는 저장되어 있을 수 있잖아?(소프트웨어 설정? 하드웨어 설정에 따라서 부분적으로 저장하고 반복하는 식인지, 한번에 저장하는 식인지는 잘 모르겠지만) 그러면 만약 처음부터 retry 한다면, 중복된 데이터가 발생할 수 있으니 멱등성(Idempotency)에서 위반/벗어나는 거지?

2. Backfill 시나리오 같은 경우에는, 좀 많이 어려우면서도 평소에 나도 이럴 땐 어떨까?라고 생각한 적은 있는데, 비즈니스 로직이 바뀐다는 게 어떻게 바뀌는 건지 모르겠네? DB의 속성(열)이 바뀌는 건가? 아니면 기존 거에 추가되는 건가? 아니면 이거는 그대로고 다른 로직이 바뀌는 건가?

3. 오.. JSON은 DB처럼 pk 명시가 보통 안 되어 있구나. 그러면 우리가 pk 기준을 세우면 되지 않을까? 음.. 로그들도 종류가 있을 거 아니야? 이런 로그들에 각각에 id를 부여해서, 이 id를 pk로 우리가 직접 설정하는 거지.
```

A-1 조정 및 새로운 Q-2.



```
1. Partial Failure와 'Atomic Write'의 마법
네 말이 정확해. Append 모드로 데이터를 쓰고 있었다면, 재시도(Retry) 시 앞서 성공한 80%에 더해 100%가 새로 쌓이면서 **총 180%의 데이터(80% 중복)**가 생기게 되지. 이게 바로 데이터 정합성이 깨지는 전형적인 케이스야.

생각해볼 점: Spark는 이 문제를 해결하기 위해 **'Temporary Directory'**라는 개념을 써. 데이터를 최종 목적지에 바로 쓰는 게 아니라, 임시 폴더에 다 쓴 뒤에 작업이 성공하면 "짠!" 하고 한 번에 옮기는 거지.

질문: 그런데 AWS S3는 일반적인 파일 시스템(HDFS)과 달리 '파일 이름 바꾸기(Rename)'가 매우 비싼 작업이거나 지원되지 않는 경우도 있어. 이런 Object Storage 환경에서 Spark가 어떻게 '한 번에 성공 혹은 실패'를 보장할 수 있을까? (힌트: S3A Committer라는 키워드를 한번 고민해봐.)

2. Backfill: 비즈니스 로직이 바뀐다는 것
실무에서 로직이 바뀐다는 건 이런 상황들이야:

계산식 변경: "기존에는 '구매' 버튼만 누르면 매출로 집계했는데, 이제부터는 '결제 완료'가 된 것만 매출로 집계해 주세요."

버그 수정: "알고 보니 지난 한 달 동안 안드로이드 유저들의 활동 로그에 특정 필드가 누락되어 잘못 계산되고 있었어요."

새로운 컬럼 추가: "기존 유저 로그에 '유저 등급' 정보를 결합(Join)해서 과거 데이터까지 다 채워 넣어 주세요."

생각해볼 점: 이때 1년 치 데이터를 다 다시 돌리는 건 돈과 시간 낭비겠지? 그래서 우리는 데이터를 '날짜(dt)' 단위로 파티셔닝해서 저장해.

질문: 만약 네가 11월 1일부터 11월 10일까지의 데이터를 새로 고쳐야 한다면, **기존의 11월 11일 이후 데이터는 건드리지 않고 해당 열흘치만 싹 갈아 끼우는 '격리된 Overwrite'**를 어떻게 구현할 수 있을까?

3. JSON에서 PK(Primary Key) 만들기
"우리가 기준을 세우면 되지 않을까?" -> 정답이야. 데이터 엔지니어는 데이터에 질서를 부여하는 사람이지.

상혁의 아이디어: 로그마다 ID를 부여하자.

시니어의 태클: 만약 모바일 앱에서 서버로 로그를 보낼 때, 네트워크 불안정으로 앱이 똑같은 로그를 두 번 보냈다고 치자. 서버는 이걸 별개의 사건으로 보고 서로 다른 ID를 부여해서 저장했어. 하지만 비즈니스적으로 이건 '하나의 클릭'이어야 해.

질문: 이럴 때 시스템이 부여한 ID 대신, 데이터 내용(유저ID, 클릭시간, 클릭한 상품ID 등)을 조합해서 고유한 키를 만들어낼 수 있을까? 이런 방식을 뭐라고 부를까? (힌트: Hash 함수)
```

새로운 A-2(My Answer)




```
1번에서 궁금한 게, 그럼 Spark의 임시 폴더에서는 100%까지 다 채워지지 못하고 80%에서 중단되었다면, 여기에 있는 데이터는 다 날라가고 다시 처음부터 쌓는 거야?

1번 질문에 대한 답은, 일단 먼저 궁금한 게 '이름 바꾸기'가 매우 비싼 작업이거나 지원되지 않는 경우가 있다는 건, 임시 폴더에서 완성된 것을 확정지어서 내 S3 폴더에 넣어서 그런 거야? 데이터 멱등성을 위해서 이렇게 일부러 설정을 했기에, 이름 조차도 변경을 허용하지 않는 건가?

너가 힌트 준 S3A 커미터(committer) 찾아봤는데, Spark 같은 빅데이터 프레임워크가 S3 객체 스토리지에 데이터를 안정적이고 효율적으로 기록(커밋)하기 위한 메커니즘이라는데, 이름 바꾸기(rename)의 성능 병목 현상을 제거하고 데이터의 일관성을 보장하는 역할을 한대.
->아마 여기서도 유추할 수 있는게, Spark -> S3로 빅데이터를 전송하는데, 이름 바꿔버리면 멱등성(처음 결과와 달라짐)이 깨져서 그런가? 그래서 막아놓는 거 같은데

너의 1번 질문에 대한 답은 음.. 몰라 일단은 rename 방지? 밖에 모르겠어.

2번의 경우에는,
데이터를 복사해서 테스트하면 좋지 않을까? 원본은 건드리지 않으면서, 복사본으로 여러번 테스트하고, 문제 생기면 버리고 다시 복사본 떠서 하고.. 이런 식으로.. 가장 정석적인? 방법 같은데..

3번의 경우에는,
단일 기본키가 아닌 복합키(2개 이상의 속성으로 이루어진 기본키)로 구성하라는 의미 같은데,
해시 함수는 딕셔너리 같은 개념 아닌가? 키-벨류
해시 함수를 찾아보니, '임의의 길이 데이터를 고정된 길이 값(해시값)으로 변환하는 단방향 함수'라는데, 내가 알기론 해시 알고리즘이 해시 함수를 이용해서 딕셔너리 형태로 만드는 걸로 알고 있는데,
즉, 해시 함수를 이용해서 해시값을 생성해서 키-벨류로 나타내는 거지.

뭐 암튼 그런 복합키를 해시함수를 통해 해시값으로 만들어서 이를 통해서 해시값이 같으면 1개로만 처리하는 건가? 그래서 멱등성 유지하는 거야?
```


A-2에 대한 조정 답안(최종)



```
1. Spark의 임시 폴더와 S3의 비밀 (Partial Failure)
"80%에서 중단되면 그 데이터는 어떻게 되나요?"

기본적으로 Spark는 작업이 실패하면 그 임시 데이터를 무시하고, 다시 실행(Retry)할 때 새로운 임시 경로를 만들거나 기존 것을 덮어써. 즉, '성공한 것만 최종 폴더로 옮긴다'는 원칙을 지키려고 노력하지.

하지만 네가 짚은 S3의 특성이 여기서 중요해:

S3는 파일 시스템이 아니라 객체 스토리지야. 우리가 보는 '폴더/디렉토리'는 사실 가짜고, 전체 경로 자체가 파일의 이름(Key)일 뿐이지.

Rename의 진실: S3에서 이름을 바꾸는 건 **'데이터 복사(Copy) + 원본 삭제(Delete)'**야. 만약 1TB 데이터를 옮겨야 한다면, 1TB를 새로 복사하는 시간만큼 시스템이 멈춰있어야 해. 게다가 복사 중에 에러가 나면? 일부는 옮겨졌고 일부는 안 옮겨진 정합성이 깨진 상태가 되어버려.

S3A Committer의 역할: 그래서 "이름 바꾸기"를 안 해. 대신 S3의 **'Multipart Upload'**라는 기능을 써서, 데이터를 '업로드 중' 상태로 보이지 않게 보관하다가, 모든 작업이 성공하면 "이제 보여줘!"라고 한 번에 신호를 보내는 거야. 이게 바로 원자성(Atomicity)을 지키는 기술이지.

2. Backfill: 복사본 테스트는 정석이지만...
"복사본으로 테스트하는 게 가장 안전하지 않나요?"

맞아, 상혁아. 개발/테스트 환경에서는 그게 정석이야. 하지만 데이터가 100PB(페타바이트)라면? 복사하는 데만 며칠이 걸리고 비용도 엄청나겠지.

실무의 해법: 그래서 우리는 **'파티션(Partition)'**을 사용해.

만약 dt=20231101, dt=20231102 처럼 날짜별로 폴더가 나뉘어 있다면, Spark의 dynamic partition overwrite 설정은 해당 날짜 폴더만 통째로 갈아 끼워.

이 방식은 원본의 다른 날짜 데이터(11월 3일~12월 31일)를 건드리지 않으면서도, 특정 날짜만 '복사본' 없이 안전하고 빠르게 수정할 수 있게 해주지.

3. JSON PK와 해시(Hash)의 마법
"복합키를 해시값으로 만들어서 중복을 제거하나요?"

정확해! 100점짜리 답변이야.

왜 해시인가? 로그 데이터는 컬럼이 100개쯤 될 수도 있어. 중복을 찾으려고 100개 컬럼의 문자열을 하나하나 비교하는 건 CPU에 엄청난 부담이지.

전략: hash(유저ID + 클릭시간 + 상품ID)를 하면 아주 짧은 고유 문자열(예: a1b2c3d4)이 나와.

이 해시값만 비교하면 훨씬 빠르게 중복을 찾아낼 수 있어. 이걸 '데드랍(Deduplication)' 과정이라고 불러.
```




---



# 1. Spark 세팅 및 데이터 생성

In [1]:
# 1. PySpark 설치
!pip install pyspark

# 2. Spark UI 등을 보기 위해 가끔 필요한 라이브러리(선택 사항)
!pip install findspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# 1. 멱등성을 위한 핵심 설정: Dynamic Partition Overwrite(위 QnA의 2번 'Backfill' 답변 확인)
spark = SparkSession.builder \
    .appName("Idempotency_Test") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic") \
    .getOrCreate()

# 2. 샘플 데이터 생성(유저 101이 10시 10분에 똑같은 행동을 2번 함)
data = [
    ("evt_001", "user_101", "2023-11-01 10:10:00", "click", "2023-11-01"),
    ("evt_002", "user_101", "2023-11-01 10:10:00", "click", "2023-11-01"), # 중복 데이터
    ("evt_003", "user_102", "2023-11-01 11:00:00", "view", "2023-11-01"),
    ("evt_004", "user_103", "2023-11-02 09:00:00", "purchase", "2023-11-02")
]

columns = ["event_id", "user_id", "timestamp", "action", "dt"]
df = spark.createDataFrame(data,columns)



# 2. Mission C - 해시 기반 중복 제거(Deduplication)

단순히 'event_id'가 다르다고 해서 다른 데이터로 보면 안된다.

비즈니스 로직에 맞춰 중복을 제거해보자.

## 2-1. spark dataframe의 dropDuplicates 함수 활용.

가장 간단하게 테스트를 해볼 수 있는 코드이다.

spark의 내장 함수를 활용해보았다.

**※문제점 발생 여지 부분**



```
1. 누가 살아남을 것인가?(who survive?)
dropDupilcates()는 중복된 데이터 중 무엇을 남길지 정할 수 없다. 만약 똑같은 클릭 로그인데 한쪽은 IP 정보가 있고, 다른 한 쪽은 IP가 누락된 채로 중복 수집되었다면?
->dropDuplicates()를 쓰면 운 좋게 정보가 있는 데이터가 남을 수도 있지만, 반대로 정보가 없는 데이터가 남을 수 있다.

2. 네트워크가 터지는 소리(Shuffle Overhead)
저 한 줄이 실행되는 순간, Spark 클러스터에서는 엄청난 일이 벌어진다.
만약 흩어져 있는 100TB 데이터라면, 'user_id','timestamp','action'이라는 기준(key)으로 한 곳에 모으는 전역 셔플(Global Shuffle)이 발생한다.
->지금이야 데이터가 4개 뿐이니 전역 셔플해도 다행이지만.. 빅데이터에서는 매우 비효율적이라는 의미.
->이럴 때는 OOM(Out of Memory)/네트워크 병목이 생기므로, 'Salting(Data Skew 현상을 해결하기 위한 방법. 동일한 id(키)에다가 랜덤한 숫자(소금)을 붙여서 강제로 찢어버리는 기법)' or 'Pre-partitioning(나중에 고생하지 말고, 저장할 때부터 미리 잘 찢어서 저장하자)' 같은 방법을 활용해야한다고 함.

3. 과거 데이터와의 싸움
지금 나의 코드는 '현재 메모리에 올라온 데이터들끼리'만 중복을 제거하고 있다.
만약 어제 들어온 데이터와 오늘 들어온 데이터가 중복이라면?
오늘 들어온 데이터만 dropDupilcates() 한다고 해서 전체 테이블의 멱등성이 보장이 될까?

```




In [3]:
# 'user_id', 'timestamp', 'action'이 같으면 동일한 행동으로 간주하고 하나만 남김
deduplicated_df = df.dropDuplicates(["user_id", "timestamp", "action"]) # spark dataframe의 dropDuplicates 함수 활용.

print(f"원본 개수: {df.count()}, 중복 제거 후: {deduplicated_df.count()}")

원본 개수: 4, 중복 제거 후: 3


## 2-2. Window Function 활용하기

진짜 실무형 중복 제거 코드로 업그레이드하기

In [4]:
from pyspark.sql.window import Window

# 예시: 윈도우 함수를 이용한 '명확한' 중복 제거
# partitionBy: GroupBy처럼 그룹화하는 건데, GroupBy와는 다르게 행의 합치지 않고, 개별 행으로 냅둠.
window_spec = Window.partitionBy("user_id","timestamp","action").orderBy(F.desc("event_id"))

# over()는 그룹 대상(윈도우)으로 보면 됨. 즉, window_spec을 윈도우(그룹 대상)로 봐서 여기에 row_number를 부여하는 것임(순차적으로 1,2,3,..) -> 번호는 건너뛰지 않음. 같은 값에 번호 건너뛰는 것은 등수 개념인 Rank()임.
# .filter("row_num == 1"): 번호표 1번(우리가 정의한 최우선 데이터)만 남기고 나머지는 버려.
# .drop("row_num"): 필터링용으로 썼던 임시 컬럼은 저장하기 전에 깔끔하게 삭제!
deduplicated_df_1 = df.withColumn("row_num", F.row_number().over(window_spec)) \
    .filter("row_num == 1") \
    .drop("row_num")

print(f"원본 개수: {df.count()}, 중복 제거 후: {deduplicated_df_1.count()}")

원본 개수: 4, 중복 제거 후: 3


**※문제점 발생 여지 부분**



```
※이 코드를 1000배 큰 데이터에 적용한다고 가정해보자.

1. 메모리 폭발: window.partitionBy는 특정 키의 데이터를 한 곳으로 모으는 셔플(shuffle)을 일으킨다.
만약, 특정 유저 한 명이 하루에 1억 번 클릭했다면, 그 유저의 데이터를 처리하는 서버에 OOM(Out of Memory)가 발생할 것이다.

2. 이에 대한 해결책: Salting 기법을 활용해서 이 윈도우 함수에 섞는 것이다.
어떻게 하면 좋을까?

-> partitionBy 안에 들어갈 컬럼에 '소금(salting)'을 쳐서, 그룹을 더 잘게 쪼개는 것이다.


```



## 2-3. 진짜 hash 함수 활용하기

Spark에서 sha2 함수를 사용해 여러 컬럼을 조합한 고유한 해시 키를 만들고, 이를 기준으로 중복을 제거하는 코드 짜보기

In [5]:
from pyspark.sql import functions as F

# 1. 해시 키 생성을 위한 컬럼 조합 (Business Key 정의)
# null 값이 섞여도 안전하게 처리하기 위해 concat_ws를 사용하는 게 실무 팁이다.
# concat_ws(concatenate(이이붙이다) with separator): 일반적인 concat() 함수는, SELECT CONCAT('First', NULL, 'Last'); 이렇게 중간에 Null이 껴있으면 전체 결과를 Null로 반환함. 하지만, concat_ws는 Null값을 무시하고 건너 뛰어서, 문자열만 이어붙임.
business_key_cols = ["user_id", "timestamp", "action"]

# 2. SHA-256 알고리즘으로 고유 지문(Hash) 생성
# 256비트 해시는 충돌(collision) 가능성이 거의 없어서 실무 pk 대용으로 자주 씀.
df_with_hash = df.withColumn(
    "row_hash",
    F.sha2(F.concat_ws("||", *[F.col(c) for c in business_key_cols]),256) # *[]: 언패킹. 즉, business_key_cols에 있는 컬럼들을 spark의 컬럼 객체로 변환하고, 리스트를 언패킹하여 concat_ws 함수 인자로 하나씩 던져준다.
    # '||'을 쓰는 이유: ||는 구분자(separator)인데, 구분자 없이 concatenate하면, ex.) user_id="12", timestamp="3"인 데이터와 user_id="1", timestamp="23"인 데이터가 둘 다 "123"이 되어버림. 이를 방지하기 위해 구분자를 쓴다.
    # F.sha2(..., 256): 합쳐진 문자열을 SHA-256 알고리즘을 통해 64글자의 고유한 16진수 값으로 변환한다.(해시값)
)

# 3. 해시값을 기준으로 중복 제거
# 이제 Spark는 100개의 컬럼을 다 안 보고, 'row_hash' 하나만 비교해서 쳐낸다 -> 멱등성 유지
deduplicated_hash_df = df_with_hash.dropDuplicates(["row_hash"])

print("--- 해시 기반 중복 제거 결과 ---")
deduplicated_hash_df.select("user_id", "timestamp", "action", "row_hash").show(truncate=False)


--- 해시 기반 중복 제거 결과 ---
+--------+-------------------+--------+----------------------------------------------------------------+
|user_id |timestamp          |action  |row_hash                                                        |
+--------+-------------------+--------+----------------------------------------------------------------+
|user_102|2023-11-01 11:00:00|view    |04474b117436a483cefe8810790b6a12e6398b9eb4e0c804d9ac2cdc0d6c6b1d|
|user_101|2023-11-01 10:10:00|click   |21cb402154abdbfc5f61c897daaa69daadea7eadcb0d08e316f857417bcfb0ce|
|user_103|2023-11-02 09:00:00|purchase|84f31d4c65e8a93e35d7333be08e39990d168615f301b9f466349b1a994a6ff5|
+--------+-------------------+--------+----------------------------------------------------------------+



**※문제점 발생 여지 부분**



```
1. 해시 충돌(Hash Collision)의 희박한 가능성
SHA-256은 매우 안전하지만, 이론적으로 서로 다른 데이터가 같은 해시값을 가질 확률이 완전히 0은 아니다.(오차가 조금이라도 있을 수 있다.)
->만약 결제 시스템처럼 단 한 건의 오차도 허용이 안 되는 곳이라면?

->해결책: 해시값으로 1차 필터링을 하되, 충돌이 의심되는 경우 실제 비즈니스 키(ID 등)를 한번 더 대조하는 방어 로직을 고려해야한다.(2차 필터링까지 설계)

2. 계산 비용 vs 저장 비용의 Trade-off(하나를 얻기 위해 다른 하나는 포기해야하는 상황)
데이터가 100PB라면, 매번 수백 개 컬럼을 합쳐 해시를 만드는 계산량(CPU)이 엄청난다. 반대로 해시값을 컬럼으로 저장하면 S3 저장 비용이 늘어난다.

->설계자의 선택: 이 파이프라인이 '자주 읽히는가(저장이 유리함)' or '한 번 쓰고 마는가(계산이 유리함)'에 따라 전략을 잘 선택해야한다.


```



# 3. Mission B - 사라진 어제의 데이터를 찾아라(Backfill & Partitioning)

※Backfill: Back + fill -> 뒤로 채워넣다.

-> 이미 지나간 과거 날짜의 데이터를 다시 처리하거나 새로 채워 넣는 작업.

## 3-1. 초기 데이터 적재(평화로운 상태)

먼저, 11월 1일,2일 데이터를 저장해보도록 하자.

In [8]:
# 1. SparkSession 생성(기본 설정 상태)
spark_1 = SparkSession.builder \
    .appName("BackfillDisaster") \
    .getOrCreate()

# 2. 11월 1일, 2일 데이터 생성
data = [
    ("evt_001", "user_A", "2023-11-01"),
    ("evt_002", "user_B", "2023-11-02")
]
columns = ["event_id", "user_id", "dt"]
df_1 = spark.createDataFrame(data, columns)

# 3. S3라고 가정하고 로컬 경로에 파티셔닝 저장
output_path = './data/log_table' # colab의 경우 content/(임시 저장소)의 data/log_table에 저장된다. 즉, /content/data/log_table
# ./data/log_table 이라고 적으면, 구글 드라이브(영구 저장소. 마운트해서 구글 드라이브랑 연결해야 함)가 아닌 현재 할당된 '가상 머신(VM)'의 임시 디스크에 저장된다. 코렙 세션이 종료되거나(런타임 초기화), 일정 시간 연결이 끊기면 여기에 저장된 데이터는 모두 삭제된다. -> 휘발성.
# colab 왼쪽 사이드바에 폴더 아이콘이 있는데, 여기서 확인 가능함.
# colab은 실행할 때마다 구글이 나에게 리눅스 컴퓨터 한 대를 빌려주는 꼴이다.

df_1.write.mode("overwrite").partitionBy("dt").parquet(output_path)
# .write: Spark의 DataFrameWriter 인터페이스 호출.
# .mode("Overwrite"): (핵심!) 저장 경로에 이미 데이터가 있다면, '덮어쓰기' 하겠다는 설정. (주의!) 기본 설정(static -> 정적)에서는 해당 '상위 폴더'를 통째로 날려버리고 새로 쓴다.
# .partitionBy("dt"): 'dt' 컬럼의 값을 기준으로 물리적인 폴더 구조를 나눔. 예: output_path/dt=2023-11-01/, output_path/dt=2023-11-02/
# parquet(output_path): 데이터를 컬럼 기반 저장 포멧인 'parquet'으로 변환하여 지정된 경로에 저장함.


print("Day 1 & Day 2 저장 완료!")
spark.read.parquet(output_path).show()
# spark.read: Spark의 DataFrameReader 인터페이스 호출.
# .parquet(output_path): 지정된 경로에서 Parquet 형식의 파일을 읽어옴.
# .show(): 앞서 정의한 읽기 과정을 실제로 실행(Action)하고, 결과를 콘솔 테이블 형태로 출력함. -> Lazy evaluation(지연 실행)(show()에서 한번에 실행.)

Day 1 & Day 2 저장 완료!
+--------+-------+----------+
|event_id|user_id|        dt|
+--------+-------+----------+
| evt_001| user_A|2023-11-01|
| evt_002| user_B|2023-11-02|
+--------+-------+----------+



## 3-2. 대재앙 발생(11월 1일만 수정하기!)

"11월 1일 데이터에 문제가 있다! 1일치만 다시 뽑아서 덮어써(Overwrite)!"라는 요청이 왔다.

In [9]:
# 1. 수정된 11월 1일 데이터
fix_data = [("evt_001_fixed", "user_A", "2023-11-01")]
fix_df = spark.createDataFrame(fix_data, columns)

# 2. ⚠️ 주의: 'overwrite' 모드로 저장 시도
fix_df.write.mode("overwrite").partitionBy("dt").parquet(output_path)

print("11월 1일 데이터 수정본 저장 완료... 과연 11월 2일 데이터는?")
try:
    spark.read.parquet(output_path).show()
except Exception as e:
    print("에러 발생 혹은 데이터 유실!")



11월 1일 데이터 수정본 저장 완료... 과연 11월 2일 데이터는?
+-------------+-------+----------+
|     event_id|user_id|        dt|
+-------------+-------+----------+
|evt_001_fixed| user_A|2023-11-01|
|      evt_002| user_B|2023-11-02|
+-------------+-------+----------+



**※Gemini의 예상**: 11월 1일 데이터(evt_001_fixed)는 잘 들어감. 하지만, 11월 2일 데이터(evt_002)는 흔적도 없이 사라졌을 것이다. static 모드 때문이다.

-> 위에서도 말한 것처럼, overwrite할 때 기본값은 **static**이다. Spark는 overwrite 명령을 받으면, 데이터를 쓰기 전에 목적지 폴더(./data/log_table) 자체를 통째로 삭제시켜버린다.

->**Dynamic Mode**(우리가 원하는 방식): 전체 폴더를 지우는 것이 아니라, 내가 지금 쓰려는 파티션(dt=2023-11-01) 폴더만 찾아서 갈아 끼워준다.


---



하지만, Gemini의 예상과는 다르게 event_02의 폴더는 사라지지 않았다.

왜 그럴까?

'**getOrCreate()**' 때문이다.

우리가 위 **대제목 1번. 'Spark 세팅 및 데이터 생성'에서 'dynamic'으로 설정했다.**

SparkSession은 싱글톤(Singleton)처럼 동작해서, **한 번 dynamic으로 설정된 세션이 살아있으면, 새로운 변수(spark_1)에 할당하더라도 기존의 dynamic 설정이 그대로 유지된 채 세션이 들어오게 된다.**

즉, 만약에 내가 1번 과정에서 dynamic을 쓰지 않았다면, 기본값인 static으로 적용돼서 에러가 났을 것이다.


# 4. Mission A - 격리된 덮어쓰기(Dynamic Partition Overwrite)

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# 1. 명시적으로 설정을 포함하여 세션 생성 (설계자의 의도 반영) -> 근데 dynamic 설정 동일함(바뀐 부분은 딱히 없음).
spark_2 = SparkSession.builder \
    .appName("MultiPartitionBackfill") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic") \
    .getOrCreate()

# 2. 초기 데이터 생성(11/1 ~ 11/5)
initial_data = [
    ("evt_1", "user_A", "2023-11-01"),
    ("evt_2", "user_B", "2023-11-02"),
    ("evt_3", "user_C", "2023-11-03"),
    ("evt_4", "user_D", "2023-11-04"),
    ("evt_5", "user_E", "2023-11-05")
]
columns = ["event_id", "user_id", "dt"]
df_initial = spark_2.createDataFrame(initial_data, columns)

output_path_1 = "./data/final_log_table"

##. 초기 적재
df_initial.write.mode("overwrite").partitionBy("dt").parquet(output_path_1)
print("--- [초기 상태] 11/01 ~ 11/05 데이터 적재 완료 ---")
spark_2.read.parquet(output_path_1).orderBy("dt").show()

# 3. 백필(Backfill) 데이터 생성(11/1, 11/3일만 수정)
backfill_data = [
    ("evt_1_fixed", "user_A", "2023-11-01"),
    ("evt_3_fixed", "user_C", "2023-11-03")
]
df_backfill = spark_2.createDataFrame(backfill_data, columns)

# 4. 격리된 덮어쓰기 실행
print("--- [백필 실행] 11/01, 11/03 데이터만 교체 중... ---")
df_backfill.write.mode("overwrite").partitionBy("dt").parquet(output_path_1)

# 5. 최종 결과 확인
print("--- [최종 결과] 과연 다른 날짜들은 무사할까? ---")
spark_2.read.parquet(output_path_1).orderBy("dt").show()


--- [초기 상태] 11/01 ~ 11/05 데이터 적재 완료 ---
+--------+-------+----------+
|event_id|user_id|        dt|
+--------+-------+----------+
|   evt_1| user_A|2023-11-01|
|   evt_2| user_B|2023-11-02|
|   evt_3| user_C|2023-11-03|
|   evt_4| user_D|2023-11-04|
|   evt_5| user_E|2023-11-05|
+--------+-------+----------+

--- [백필 실행] 11/01, 11/03 데이터만 교체 중... ---
--- [최종 결과] 과연 다른 날짜들은 무사할까? ---
+-----------+-------+----------+
|   event_id|user_id|        dt|
+-----------+-------+----------+
|evt_1_fixed| user_A|2023-11-01|
|      evt_2| user_B|2023-11-02|
|evt_3_fixed| user_C|2023-11-03|
|      evt_4| user_D|2023-11-04|
|      evt_5| user_E|2023-11-05|
+-----------+-------+----------+



※시니어 멘토(Gemini)의 '결과 검증' 질문



```
1. 데이터 보존: dynamic 설정 덕분에 11/2,4,5일 데이터가 잘 살아있음을 알 수 있다.

2. 물리적 구조: 격리된 업데이트(밑에 참고) 때문에 각 폴더의 생성 시간이 다르다. 이게 왜 'S3 저장 비용과 네트워크 비용을 아껴줄 수 있을까?

-> A. 만약, 내 데이터가 10년치(약 3650개 파티션)라고 가정해보자.

＊static 모드(비효율): 10년치 폴더를 전부 삭제하고, 다시 10년치를 써야한다.
(만약 내가 10년 치 데이터를 메모리에 다 들고 있지 않다면, S3에서 10년 치를 읽어와서 다시 쓰는 과정에서 엄청난 '네트워크 대역폭'을 소모하고, 'S3 쓰기 요청(PUT Request)'이 폭발한다.

＊Dynamic 모드(효율): Spark는 딱 수정이 필요한 2개 파티션의 데이터만 네트워크로 실어 보낸다. 나머지 3648개의 폴더는 아예 건드리지도 않는다.
-> 네트워크: 전송 데이터 용량이 획기적으로 줄어듦.
-> 비용: S3 API 호출 횟수와 데이터 전송료(Egress)가 최소화됨.

3. 원자성(Atomicity): 만약 11월 1일과 3일을 동시에 덮어쓰다가, 3일치 데이터를 쓰는 도중에 에러가 났다. S3A committer가 작동한다면, 이미 성공한 1일치 데이터는 어떻게 될까?(다 같이 취소될까, 아니면 1일치만 바뀔까?)

나는 위 S3A committer의 특징 상, 성공한 것만(최종 성공) 최종 폴더로 옮긴다고 하고, 임시 폴더에서 retry 하는 것들은 다시 지우고 처음부터 다시 한다고 했으니, 1일치만 바뀌고 3일치는 다시 할 거 같다고 생각했다.

하지만 틀렸다고 한다.

만약 그렇게 작동한다면 이건 '완벽한 멱등성'이 아니라고 한다.

데이터 엔지니어링에서 작업(Job)은 '전부 성공하거나 아예 실패하거나(All or Nothing)'이여야 한다고 한다.

3-1. Partial(부분) Update의 위험: 11월 1일은 수정됐는데, 3일은 안 됐다면, 관리자는 '이 작업이 성공한 건가? 실패한 건가?'라는 혼란에 빠지게 된다. 데이터 정합성(여러 데이터 값이 서로 모순 없이 일치하고, 일관된 상태를 유지하는 것)이 깨진 상태로 서비스가 운영되는 것이다.

3-2. S3A committer의 진짜 마법(Job-level):
＊11/1일치 작업이 끝나도 바로 최종 폴더로 옮기지 않는다. 임시 폴더(_temporary)에 고이 모셔둔다.
＊11/3일치까지 모든 파티션의 쓰기 작업이 완벽히 성공했을 때만, Job committer가 '자,이제 다 같이 이사 가자!'라고 신호를 보낸다.
＊만약 3일치에서 에러가 난다면, 1일치 임시 데이터도 다 같이 쓰레기통으로 가고, 원본 데이터(수정 전 11/1,11/3)는 단 한 글자도 바뀌지 않은 채로 유지된다.
```

마치, **DB의 트랜잭션**(transaction, 한 번에 모두 처리하기. 모두 성공하면 성공, 하나라도 실패하면 실패) 같은 구조네.




In [11]:
!ls -l --time-style=full-iso ./data/final_log_table # (리눅스 터미널 명령어) 각 dt=YYYY-MM-DD 폴더 옆에 마지막 수정 시간이 나온다.

total 20
drwxr-xr-x 2 root root 4096 2026-01-09 15:24:35.714739045 +0000 'dt=2023-11-01'
drwxr-xr-x 2 root root 4096 2026-01-09 15:24:33.270543456 +0000 'dt=2023-11-02'
drwxr-xr-x 2 root root 4096 2026-01-09 15:24:35.814747047 +0000 'dt=2023-11-03'
drwxr-xr-x 2 root root 4096 2026-01-09 15:24:33.102530011 +0000 'dt=2023-11-04'
drwxr-xr-x 2 root root 4096 2026-01-09 15:24:33.243541295 +0000 'dt=2023-11-05'


격리된 업데이트를 확인할 수 있다.

->백필을 진행한 11-01, 11-03 폴더의 시간은 방금 전 시간으로 업데이트되어 있지만, 건드리지 않은 11-02 등의 폴더는 처음 생성했던 과거 시간 그대로 멈춰있다.



# 5. 마무리 및 결론

이렇게 해서 이번에는, '**Perfect Idempotency(완벽한 멱등성)**'에 대해서 여러 실습을 진행해보았다.

여러 실습을 통해 멱등성이라는 개념을 실무/실습에 적용해서 더 자세하게 이해하고 알아가는 시간이 되었다.

또한 마지막에, 데이터 엔지니어링은 '**All or nothing**' 기법을 활용한다는 것도 새롭게 배워갔다.



---
다음엔 실무 Exercise의 마지막인, '**Data Trust(데이터 신뢰성)**'으로 돌아오도록 하겠다.
